In [ ]:
# Imports des librairies

import pandas as pd
import time
from nba_api.stats.endpoints import LeagueDashTeamStats

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
# Liste des saisons 2000-01 à 2024-25

saisons = [f"{annee}-{str(annee + 1)[-2:]}" for annee in range(2000, 2025)]
print(f"{len(saisons)} saisons : {saisons[0]} → {saisons[-1]}")

25 saisons : 2000-01 → 2024-25


In [ ]:
# Extraction Base + Advanced pour PACE

liste_base = []
liste_advanced = []
saisons_echouees = []

for saison in saisons:
    try:
        ldts_base = LeagueDashTeamStats(season=saison, measure_type_detailed_defense='Base', timeout=60)
        df_base = ldts_base.get_data_frames()[0]
        df_base["SEASON"] = saison
        liste_base.append(df_base)

        ldts_adv = LeagueDashTeamStats(season=saison, measure_type_detailed_defense='Advanced', timeout=60)
        df_adv = ldts_adv.get_data_frames()[0]
        df_adv["SEASON"] = saison
        liste_advanced.append(df_adv)

        print(f">>> {saison} <<<")
    except Exception as e:
        saisons_echouees.append(saison)
        print(f"Échec {saison} : {e}")
    time.sleep(2)

>>> 2000-01 <<<
>>> 2001-02 <<<
>>> 2002-03 <<<
>>> 2003-04 <<<
>>> 2004-05 <<<
>>> 2005-06 <<<
>>> 2006-07 <<<
>>> 2007-08 <<<
>>> 2008-09 <<<
>>> 2009-10 <<<
>>> 2010-11 <<<
>>> 2011-12 <<<
>>> 2012-13 <<<
>>> 2013-14 <<<
>>> 2014-15 <<<
>>> 2015-16 <<<
>>> 2016-17 <<<
>>> 2017-18 <<<
>>> 2018-19 <<<
>>> 2019-20 <<<
>>> 2020-21 <<<
>>> 2021-22 <<<
>>> 2022-23 <<<
>>> 2023-24 <<<
>>> 2024-25 <<<


In [ ]:
# Fusion des deux DF

colonnes_adv = ["TEAM_ID", "SEASON", "POSS", "PACE", "PACE_PER40", "E_PACE", "OFF_RATING", "DEF_RATING", "NET_RATING", "EFG_PCT", "TS_PCT"]

df_base_final = pd.concat(liste_base, ignore_index=True) if liste_base else pd.DataFrame()
df_adv_final = pd.concat(liste_advanced, ignore_index=True) if liste_advanced else pd.DataFrame()

if not df_base_final.empty and not df_adv_final.empty:
    df_adv_subset = df_adv_final[colonnes_adv]
    df_final = df_base_final.merge(df_adv_subset, on=["TEAM_ID", "SEASON"], how="left")
    df_final.to_csv("nba_team_stats_2000_2025.csv", index=False)
    print(f"Exporté : {len(df_final)} lignes, {len(df_final.columns)} colonnes")

if saisons_echouees:
    print(f"Saisons en échec : {saisons_echouees}")

Exporté : 746 lignes, 64 colonnes


In [ ]:
# Vérification des colonnes

df_final

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,SEASON,POSS,PACE,PACE_PER40,E_PACE,OFF_RATING,DEF_RATING,NET_RATING,EFG_PCT,TS_PCT
0,1610612737,Atlanta Hawks,82,25,57,0.305,3946.000000,2876,6668,0.431,333,933,0.357,1374,1811,0.759,1028,2490,3518,1559,1368.0,634,387,513,1863,7,7459,-427.0,1,25,25,25,27,20,12,22,20,24,11,27,27,13,9,16,12,28,28,16,21,27,14,8,26,25,2000-01,7695,93.63,78.02,94.5,96.9,102.4,-5.5,0.456,0.500
1,1610612738,Boston Celtics,82,36,46,0.439,3966.000000,2773,6485,0.428,592,1633,0.363,1621,2190,0.740,897,2367,3264,1708,1285.0,769,336,440,1952,3,7759,-175.0,1,20,20,20,13,26,20,25,1,1,8,6,7,20,25,27,26,19,19,3,28,17,25,22,16,20,2000-01,7748,93.81,78.17,95.0,100.1,102.3,-2.2,0.473,0.521
2,1610612766,Charlotte Hornets,82,46,36,0.561,3976.000000,2800,6501,0.431,340,984,0.346,1599,2146,0.745,1033,2608,3641,1900,1183.0,665,455,375,1753,5,7539,172.0,1,14,14,14,8,24,17,23,19,20,18,9,8,19,8,4,7,7,8,11,11,6,11,18,24,14,2000-01,7565,91.24,76.04,92.0,99.7,97.6,2.1,0.457,0.506
3,1610612741,Chicago Bulls,82,15,67,0.183,3971.000000,2721,6411,0.424,329,950,0.346,1410,1909,0.739,926,2260,3186,1810,1292.0,675,379,426,1902,3,7181,-746.0,1,29,29,29,10,28,25,27,22,22,17,26,22,21,20,29,29,12,21,7,24,14,19,22,29,29,2000-01,7486,90.47,75.39,91.7,95.9,105.9,-10.0,0.450,0.495
4,1610612739,Cleveland Cavaliers,82,30,52,0.366,3965.613333,2890,6532,0.442,220,659,0.334,1561,2040,0.765,1015,2440,3455,1708,1350.0,642,436,542,1904,8,7561,-348.0,1,23,23,23,17,18,16,14,29,29,24,12,14,8,11,20,17,19,27,15,13,29,20,6,22,23,2000-01,7613,92.32,76.94,93.9,99.3,103.5,-4.2,0.459,0.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,1610612758,Sacramento Kings,82,40,42,0.488,3976.000000,3529,7385,0.478,1032,2887,0.357,1400,1738,0.806,901,2723,3624,2169,1092.0,620,357,350,1548,1515,9490,39.0,1,16,16,16,2,9,10,12,23,24,19,13,19,4,16,14,16,13,6,25,25,5,17,19,10,16,2024-25,8189,98.91,82.42,100.6,115.9,115.3,0.6,0.548,0.582
742,1610612759,San Antonio Spurs,82,34,48,0.415,3946.000000,3426,7362,0.465,1158,3246,0.357,1332,1721,0.774,896,2685,3581,2348,1138.0,674,455,356,1363,1428,9342,-226.0,1,23,23,23,23,14,15,17,9,7,20,24,21,21,17,21,19,6,10,12,4,7,3,28,16,21,2024-25,8231,100.08,83.40,101.9,113.5,116.3,-2.8,0.544,0.575
743,1610612761,Toronto Raptors,82,30,52,0.366,3961.000000,3414,7462,0.458,969,2786,0.348,1294,1729,0.748,1034,2666,3700,2340,1257.0,664,348,484,1738,1484,9091,-352.0,1,24,24,24,9,15,6,20,29,28,23,28,20,29,4,24,10,7,24,16,28,29,30,24,23,24,2024-25,8297,100.62,83.85,102.4,109.6,113.6,-4.1,0.522,0.553
744,1610612762,Utah Jazz,82,17,65,0.207,3961.000000,3305,7272,0.454,1144,3267,0.350,1421,1836,0.774,983,2741,3724,2093,1412.0,555,360,527,1549,1580,9175,-766.0,1,30,30,30,9,25,20,22,10,6,22,8,10,22,8,11,5,20,30,30,24,30,18,4,21,28,2024-25,8322,100.85,84.04,102.7,110.2,119.4,-9.2,0.533,0.568


In [ ]:
# Suppression des colonnes de rang (inutiles pour objectifs)

df_final = df_final.loc[:, ~df_final.columns.str.endswith('_RANK')]
df_final

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON,POSS,PACE,PACE_PER40,E_PACE,OFF_RATING,DEF_RATING,NET_RATING,EFG_PCT,TS_PCT
0,1610612737,Atlanta Hawks,82,25,57,0.305,3946.000000,2876,6668,0.431,333,933,0.357,1374,1811,0.759,1028,2490,3518,1559,1368.0,634,387,513,1863,7,7459,-427.0,2000-01,7695,93.63,78.02,94.5,96.9,102.4,-5.5,0.456,0.500
1,1610612738,Boston Celtics,82,36,46,0.439,3966.000000,2773,6485,0.428,592,1633,0.363,1621,2190,0.740,897,2367,3264,1708,1285.0,769,336,440,1952,3,7759,-175.0,2000-01,7748,93.81,78.17,95.0,100.1,102.3,-2.2,0.473,0.521
2,1610612766,Charlotte Hornets,82,46,36,0.561,3976.000000,2800,6501,0.431,340,984,0.346,1599,2146,0.745,1033,2608,3641,1900,1183.0,665,455,375,1753,5,7539,172.0,2000-01,7565,91.24,76.04,92.0,99.7,97.6,2.1,0.457,0.506
3,1610612741,Chicago Bulls,82,15,67,0.183,3971.000000,2721,6411,0.424,329,950,0.346,1410,1909,0.739,926,2260,3186,1810,1292.0,675,379,426,1902,3,7181,-746.0,2000-01,7486,90.47,75.39,91.7,95.9,105.9,-10.0,0.450,0.495
4,1610612739,Cleveland Cavaliers,82,30,52,0.366,3965.613333,2890,6532,0.442,220,659,0.334,1561,2040,0.765,1015,2440,3455,1708,1350.0,642,436,542,1904,8,7561,-348.0,2000-01,7613,92.32,76.94,93.9,99.3,103.5,-4.2,0.459,0.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,1610612758,Sacramento Kings,82,40,42,0.488,3976.000000,3529,7385,0.478,1032,2887,0.357,1400,1738,0.806,901,2723,3624,2169,1092.0,620,357,350,1548,1515,9490,39.0,2024-25,8189,98.91,82.42,100.6,115.9,115.3,0.6,0.548,0.582
742,1610612759,San Antonio Spurs,82,34,48,0.415,3946.000000,3426,7362,0.465,1158,3246,0.357,1332,1721,0.774,896,2685,3581,2348,1138.0,674,455,356,1363,1428,9342,-226.0,2024-25,8231,100.08,83.40,101.9,113.5,116.3,-2.8,0.544,0.575
743,1610612761,Toronto Raptors,82,30,52,0.366,3961.000000,3414,7462,0.458,969,2786,0.348,1294,1729,0.748,1034,2666,3700,2340,1257.0,664,348,484,1738,1484,9091,-352.0,2024-25,8297,100.62,83.85,102.4,109.6,113.6,-4.1,0.522,0.553
744,1610612762,Utah Jazz,82,17,65,0.207,3961.000000,3305,7272,0.454,1144,3267,0.350,1421,1836,0.774,983,2741,3724,2093,1412.0,555,360,527,1549,1580,9175,-766.0,2024-25,8322,100.85,84.04,102.7,110.2,119.4,-9.2,0.533,0.568


In [ ]:
# Suppression de colonnes inutiles supplémentaires

df_final = df_final.drop(columns=['TEAM_ID', 'E_PACE'])
df_final

,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON,POSS,PACE,PACE_PER40,OFF_RATING,DEF_RATING,NET_RATING,EFG_PCT,TS_PCT
0,Atlanta Hawks,82,25,57,0.305,3946.000000,2876,6668,0.431,333,933,0.357,1374,1811,0.759,1028,2490,3518,1559,1368.0,634,387,513,1863,7,7459,-427.0,2000-01,7695,93.63,78.02,96.9,102.4,-5.5,0.456,0.500
1,Boston Celtics,82,36,46,0.439,3966.000000,2773,6485,0.428,592,1633,0.363,1621,2190,0.740,897,2367,3264,1708,1285.0,769,336,440,1952,3,7759,-175.0,2000-01,7748,93.81,78.17,100.1,102.3,-2.2,0.473,0.521
2,Charlotte Hornets,82,46,36,0.561,3976.000000,2800,6501,0.431,340,984,0.346,1599,2146,0.745,1033,2608,3641,1900,1183.0,665,455,375,1753,5,7539,172.0,2000-01,7565,91.24,76.04,99.7,97.6,2.1,0.457,0.506
3,Chicago Bulls,82,15,67,0.183,3971.000000,2721,6411,0.424,329,950,0.346,1410,1909,0.739,926,2260,3186,1810,1292.0,675,379,426,1902,3,7181,-746.0,2000-01,7486,90.47,75.39,95.9,105.9,-10.0,0.450,0.495
4,Cleveland Cavaliers,82,30,52,0.366,3965.613333,2890,6532,0.442,220,659,0.334,1561,2040,0.765,1015,2440,3455,1708,1350.0,642,436,542,1904,8,7561,-348.0,2000-01,7613,92.32,76.94,99.3,103.5,-4.2,0.459,0.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,Sacramento Kings,82,40,42,0.488,3976.000000,3529,7385,0.478,1032,2887,0.357,1400,1738,0.806,901,2723,3624,2169,1092.0,620,357,350,1548,1515,9490,39.0,2024-25,8189,98.91,82.42,115.9,115.3,0.6,0.548,0.582
742,San Antonio Spurs,82,34,48,0.415,3946.000000,3426,7362,0.465,1158,3246,0.357,1332,1721,0.774,896,2685,3581,2348,1138.0,674,455,356,1363,1428,9342,-226.0,2024-25,8231,100.08,83.40,113.5,116.3,-2.8,0.544,0.575
743,Toronto Raptors,82,30,52,0.366,3961.000000,3414,7462,0.458,969,2786,0.348,1294,1729,0.748,1034,2666,3700,2340,1257.0,664,348,484,1738,1484,9091,-352.0,2024-25,8297,100.62,83.85,109.6,113.6,-4.1,0.522,0.553
744,Utah Jazz,82,17,65,0.207,3961.000000,3305,7272,0.454,1144,3267,0.350,1421,1836,0.774,983,2741,3724,2093,1412.0,555,360,527,1549,1580,9175,-766.0,2024-25,8322,100.85,84.04,110.2,119.4,-9.2,0.533,0.568


In [ ]:
# Réorganisation des colonnes pour SEASON first (lisibilité DF)

cols = ['SEASON'] + [col for col in df_final.columns if col != 'SEASON']
df_final = df_final[cols]
df_final

,SEASON,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,POSS,PACE,PACE_PER40,OFF_RATING,DEF_RATING,NET_RATING,EFG_PCT,TS_PCT
0,2000-01,Atlanta Hawks,82,25,57,0.305,3946.000000,2876,6668,0.431,333,933,0.357,1374,1811,0.759,1028,2490,3518,1559,1368.0,634,387,513,1863,7,7459,-427.0,7695,93.63,78.02,96.9,102.4,-5.5,0.456,0.500
1,2000-01,Boston Celtics,82,36,46,0.439,3966.000000,2773,6485,0.428,592,1633,0.363,1621,2190,0.740,897,2367,3264,1708,1285.0,769,336,440,1952,3,7759,-175.0,7748,93.81,78.17,100.1,102.3,-2.2,0.473,0.521
2,2000-01,Charlotte Hornets,82,46,36,0.561,3976.000000,2800,6501,0.431,340,984,0.346,1599,2146,0.745,1033,2608,3641,1900,1183.0,665,455,375,1753,5,7539,172.0,7565,91.24,76.04,99.7,97.6,2.1,0.457,0.506
3,2000-01,Chicago Bulls,82,15,67,0.183,3971.000000,2721,6411,0.424,329,950,0.346,1410,1909,0.739,926,2260,3186,1810,1292.0,675,379,426,1902,3,7181,-746.0,7486,90.47,75.39,95.9,105.9,-10.0,0.450,0.495
4,2000-01,Cleveland Cavaliers,82,30,52,0.366,3965.613333,2890,6532,0.442,220,659,0.334,1561,2040,0.765,1015,2440,3455,1708,1350.0,642,436,542,1904,8,7561,-348.0,7613,92.32,76.94,99.3,103.5,-4.2,0.459,0.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,2024-25,Sacramento Kings,82,40,42,0.488,3976.000000,3529,7385,0.478,1032,2887,0.357,1400,1738,0.806,901,2723,3624,2169,1092.0,620,357,350,1548,1515,9490,39.0,8189,98.91,82.42,115.9,115.3,0.6,0.548,0.582
742,2024-25,San Antonio Spurs,82,34,48,0.415,3946.000000,3426,7362,0.465,1158,3246,0.357,1332,1721,0.774,896,2685,3581,2348,1138.0,674,455,356,1363,1428,9342,-226.0,8231,100.08,83.40,113.5,116.3,-2.8,0.544,0.575
743,2024-25,Toronto Raptors,82,30,52,0.366,3961.000000,3414,7462,0.458,969,2786,0.348,1294,1729,0.748,1034,2666,3700,2340,1257.0,664,348,484,1738,1484,9091,-352.0,8297,100.62,83.85,109.6,113.6,-4.1,0.522,0.553
744,2024-25,Utah Jazz,82,17,65,0.207,3961.000000,3305,7272,0.454,1144,3267,0.350,1421,1836,0.774,983,2741,3724,2093,1412.0,555,360,527,1549,1580,9175,-766.0,8322,100.85,84.04,110.2,119.4,-9.2,0.533,0.568


In [ ]:
# Vérification du nombre d'équipes par saison

df_final['SEASON'].value_counts()

SEASON
2004-05    30
2005-06    30
2006-07    30
2007-08    30
2008-09    30
2009-10    30
2010-11    30
2011-12    30
2012-13    30
2013-14    30
2014-15    30
2015-16    30
2016-17    30
2017-18    30
2018-19    30
2019-20    30
2020-21    30
2021-22    30
2022-23    30
2023-24    30
2024-25    30
2000-01    29
2001-02    29
2002-03    29
2003-04    29
Name: count, dtype: int64

In [ ]:
# Suppression des saisons a 29 équipes (2000-01, 2001-02, 2002-03, 2003-04)

